In [ ]:
# Intended to address throughly the basics of statistics required for the course.
#Generalised Linear Models; the general linear model and the least-squares method, 
#logistic regression for binary responses, Poisson regression for count data. 
#More broadly, how to build a flexible linear predictor to capture relationships of interest. 

#Statistical Methods; commonly used probability distributions, 
#parameter estimation, sampling variability, hypothesis testing, 
#basic measures of bivariate relationships. 

library(tidyverse)
library(ggplot2)
library(aod)
library(reshape2)
library(lattice)
library(leaps)
library(vcd)
library(plotROC)
library(data.table)
require(boot)
library(broom)
require(sandwich)
require(msm)
library(ggExtra)
library(gridExtra)
list.files(path = "../input")

In [ ]:
df_cancer <- read.csv("../input/breast-cancer-wisconsin-data/data.csv", header=TRUE)
head(df_cancer)
df_cancer$diagnosis <- ifelse(df_cancer$diagnosis == "M", 1, 0)
attach(df_cancer) #make this 'active' so we can refer to the columns without df_cancer$ in front

In [ ]:
#33 columns
length(df_cancer)
#569 rows
nrow(df_cancer)

In [ ]:
#radius is explained by texture_mean, perimeter_mean, area_mean and smoothness_mean
breast_cancer_lm <- lm(radius_mean ~ texture_mean + 
                       perimeter_mean + area_mean + 
                       smoothness_mean)
summary(breast_cancer_lm)
anova(breast_cancer_lm)
cancer_data_subset <- df_cancer[c("radius_mean", "texture_mean", "perimeter_mean",
                                 "area_mean", "smoothness_mean")]

The anova above tells us (as expected) that the means we have tested are from different samples.

In [ ]:
head(cancer_data_subset)

<b>Does the radius mean predict the smoothness mean?</b><br>
Note the important geom_smooth here, method=lm refers to a linear model to be used to add a predictive linear regression line.

In [ ]:
scatter <- 
ggplot(cancer_data, aes(radius_mean, smoothness_mean)) + 
geom_point(color = "#606060") +
geom_smooth(method=lm, color="#608793", se=FALSE) +
labs(x = "Radius Mean", y = "Smoothness Mean")

scatter_with_marg <- ggExtra::ggMarginal(scatter, color = "#606060", 
                                         fill = "#606060", type = "histogram")
scatter_with_marg

<b>Correlation Matrix</b>

In [ ]:
results <- cor(cancer_data_subset)
results <- as.matrix(results)
results <- round(results, 2)

ordered_data <- function(arrange_data) {
    distance <- as.dist(1-arrange_data/2)
    clustered <- hclust(distance)
    arrange_data <- arrange_data[clustered$order, clustered$order]
}

correlation_matrix <- ordered_data(results)
correlation_matrix

melted_correlation <- melt(correlation_matrix, na.rm = TRUE)

ggheatmap <- ggplot(melted_correlation, aes(Var2, Var1, fill = value)) + 
geom_tile(color = "#262626") + 
scale_fill_gradient2(low = "#dd6546", high = "#608793", 
                     mid = "white", midpoint = 0, limit = c(-1,1), 
                     space = "Lab", name="Pearson\nCoefficient") + 
theme_minimal()

# plot heatmap and formatting
heatmap_tiles <-
ggheatmap + 
geom_text(aes(label = value), color = "#262626", size = 3) + 
coord_fixed() + 
theme(
  legend.justification = c(0, 1),
  legend.direction = "vertical",
  axis.text.x = element_text(angle = 45, size=8, vjust = 1, hjust = 1)) +
guides(fill = guide_colorbar(barwidth = 1, barheight = 7,
            title.position = "top", title.hjust = 0.5)) +
labs(title="Heatmap of Variables", x="", y="")

In [ ]:
residual_data <- augment(breast_cancer_lm)
residuals_plot <- ggplot(residual_data, aes(x = .fitted, y = .resid, label="hi")) + 
geom_point(color="#606060") + 
geom_hline(yintercept=0, linetype="dashed", color="#606060", size=0.5) +
theme_minimal() +
labs(title="Residuals", x="", y="")
res_with_marginal <- ggExtra::ggMarginal(residuals_plot, color = "#606060", 
                                         fill = "#606060", type = "histogram")

In [ ]:
qqplot <- ggplot(breast_cancer_lm, aes(sample=df_cancer$radius_mean)) + 
stat_qq_line(color="#608793") +
stat_qq(color="#606060") + 
theme_minimal() +
labs(title="QQ-plot", x="", y="")

In [ ]:
grid.arrange(arrangeGrob(qqplot, res_with_marginal, ncol=2), heatmap_tiles, nrow = 2)

<b>Logistic Regression</b>

In [ ]:
overview_plot <-
ggplot(data=df_cancer, aes(x=radius_mean, y=diagnosis)) + 
geom_point(col = "#606060") + 
theme_minimal() +
labs(title="Diagnosis versus Mean Radius", subtitle="A quick overview of the data")

In [ ]:
logit_model <- glm(diagnosis ~ radius_mean, data = df_cancer, family = "binomial")
predProbs = predict(logit_model, type="response")
summary(logit_model)

In [ ]:
logit_fit <-vggplot(data=df_cancer, 
        aes(x=radius_mean, y=diagnosis)) + 
geom_point(col = "#606060", size = 0.05) + 
theme_minimal() +
stat_smooth(method="glm", method.args=list(family="binomial"), 
            color = "#608793", fill = "#608793", se=FALSE, size = 0.5) +
labs(title="Diagnosis versus Mean Radius", subtitle="Fitted Logistic Curve",
    x = "Mean Radius", y = "Log Odds of Diagnosis")

In [ ]:
dependent_var <- diagnosis
independent_var <- radius_mean

test <- data.frame(dependent_var, independent_var)

receiver_operating_curve <- ggplot(test, aes(d = dependent_var, m = independent_var)) + 

style_roc() +

geom_abline(intercept=0, slope=1, color = "#606060") +

geom_roc(color = "#608793", labels = FALSE, size = 0.5) +

labs(title = "Receiver Operating Curve", subtitle = "Analysis of Predictive Ability") +

theme_minimal()

In [ ]:
grid.arrange(arrangeGrob(logit_fit, receiver_operating_curve, ncol=2), nrow = 2)

<b>Poisson Distribution</b>

In [ ]:
id <- c(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13)
count <- c(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 17, 24)
fraction <- c(0.11, 0.9, 0.9, 0.9, 0.9, 0.9, 0.6, 0.5, 0.5, 0.4, 0.3, 0.3, 0.2)
count_data <- data.frame(id, count, fraction)

In [ ]:
count_data

In [ ]:
count_data <- within(count_data, {
  id <- factor(id)
})
summary(count_data)

In [ ]:
ggplot(count_data, aes(fraction)) + 
geom_histogram(bins=13, fill="#608793") + 
scale_x_log10() + 
theme_minimal()

In [ ]:
summary(poisson_model <- glm(count ~ fraction, family="poisson", data=count_data))

<b>General Notes</b><br>
The goal of a <b>t-test</b> is to compare the means (from different distributions) to see if they are significantly different from each other.<br>
<b>ANOVA</b> is used to see if data points all belong to the same category